In [41]:
# Data manipulation
import numpy as np
import pandas as pd

# Web scraping
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# Other
import re
import time

# Web scraping REGDOCS

## Companies

In [64]:
# Setup chrome driver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("C:/chromedriver/chromedriver", options=options)

In [43]:
# Get list of all companies
def scrapeCompanies():
    elems = driver.find_elements_by_tag_name('a')
    companies = []

    for i in range(len(elems)):
        if ('Item/View' in elems[i].get_attribute('href')):
            companies.append(elems[i].text)
    
    return companies

In [6]:
# Scrape all companies

PATH_REGDOCS = 'https://apps.cer-rec.gc.ca/REGDOCS/Search/Companies'

driver.get(PATH_REGDOCS)

# Let page load
time.sleep(5)

# Set list to 200
itemsSelect = Select(driver.find_element_by_name('tableResults_length'))
itemsSelect.select_by_value('200')
time.sleep(5)

companies = ['Canada Energy Regulator']

for i in range(10):
    companies += scrapeCompanies()
    
    try:
        nextButton = driver.find_element_by_class_name('next')
        nextButton.click()
        time.sleep(5)
    except:
        break

In [30]:
# Load
companies_raw_df = pd.read_csv('Dataset/companies_raw.csv')

In [29]:
# Save 
companies_raw_df = pd.DataFrame({'Company': companies})
companies_raw_df.to_csv('Dataset/companies_raw.csv')

In [32]:
# Preprocess company names

companies_preprocess = []

for company in companies:
    result = ''.join(filter(lambda s: not str.isnumeric(s), company))
    result = result.strip()
    companies_preprocess.append(result)
    
# Remove duplicates
companies_preprocess = list(set(companies_preprocess))
companies_preprocess.sort()
    
# Save
companies_preprocess_df = pd.DataFrame({'Company': companies_preprocess})
companies_preprocess_df.to_csv('Dataset/companies.csv')

## Download PDF documents

In [69]:
# Setup chrome driver
options = webdriver.ChromeOptions()

options.add_experimental_option('prefs',  {
    "download.default_directory": 'data/',
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.plugins_disabled": ["Chrome PDF Viewer"],
    "plugins.always_open_pdf_externally": True
    }
)

driver = webdriver.Chrome("C:/chromedriver/chromedriver", options=options)

In [66]:
# Get urls
pdfLinks = pd.read_csv('Dataset/pdfLinks.csv')

In [72]:
# Download all

for link in pdfLinks['link']:
    driver.get(link)